<a href="https://colab.research.google.com/github/chlitsas/ml-basketball/blob/master/basketball2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sys
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from random import random
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [3]:
raw_teams = pd.read_csv(
    'https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Teams.csv',
    names=['conference', 'team_name']
    )
print(raw_teams)

raw_games = pd.read_csv(
    'https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Games.csv',
    names=['date', 'home_team', 'home_pts', 'road_team', 'road_pts']
    )
raw_games['date'] =  pd.to_datetime(raw_games['date'], format='%Y-%m-%d')

print(raw_games)

           conference              team_name
0        America East                Vermont
1        America East            Stony Brook
2        America East                   UMBC
3        America East               Hartford
4        America East                 Albany
..                ...                    ...
348  Western Athletic     California Baptist
349  Western Athletic  Cal State Bakersfield
350  Western Athletic                Seattle
351  Western Athletic                   UMKC
352  Western Athletic          Chicago State

[353 rows x 2 columns]
            date      home_team  home_pts                  road_team  road_pts
0     2015-11-13         Hawaii        87                Montana St.        76
1     2015-11-13  Eastern Mich.        70                    Vermont        50
2     2015-11-13       Columbia       107                       Kean        62
3     2015-11-13     La.-Monroe        88                    McMurry        43
4     2015-11-13           Yale        70

In [105]:
games = raw_games.dropna().copy()
teams = raw_teams.dropna().copy()

games['home_team'] = games['home_team'].str.strip()
games['road_team'] = games['road_team'].str.strip()
games['home_team'] = games['home_team'].str.replace('&#039;','\'')
games['road_team'] = games['road_team'].str.replace('&#039;','\'')
games['home_team'] = games['home_team'].str.replace('&amp;','&')
games['road_team'] = games['road_team'].str.replace('&amp;','&')
games['home_team'] = games['home_team'].str.replace('N.J.','NJ')
games['road_team'] = games['road_team'].str.replace('N.J.','NJ')
games['home_team'] = games['home_team'].str.replace('N.Y.','NY')
games['road_team'] = games['road_team'].str.replace('N.Y.','NY')
games['home_team'] = games['home_team'].str.replace('S\'west','Southwest')
games['road_team'] = games['road_team'].str.replace('S\'west','Southwest')
games['home_team'] = games['home_team'].str.replace(' St.',' State')
games['road_team'] = games['road_team'].str.replace(' St.',' State')
games['home_team'] = games['home_team'].str.replace('Eastern Mich.','Eastern Michigan')
games['road_team'] = games['road_team'].str.replace('Eastern Mich.','Eastern Michigan')
games['home_team'] = games['home_team'].str.replace('Central Mich.','Central Michigan')
games['road_team'] = games['road_team'].str.replace('Central Mich.','Central Michigan')
games['home_team'] = games['home_team'].str.replace('La.-Monroe','Louisiana-Monroe')
games['road_team'] = games['road_team'].str.replace('La.-Monroe','Louisiana-Monroe')
games['home_team'] = games['home_team'].str.replace('Penn','Penn State')
games['road_team'] = games['road_team'].str.replace('Penn','Penn State')
games['home_team'] = games['home_team'].str.replace('St. John\'s (NY)','St. John\'s')
games['road_team'] = games['road_team'].str.replace('St. John\'s (NY)','St. John\'s')

teams['team_name'] = teams['team_name'].str.strip()



assert len(games) == games['home_team'].count()
assert len(games) == games['road_team'].count()
assert len(teams) == teams['team_name'].count()
print(len(games))

23492


In [109]:
from google.colab import drive
#drive.mount('drive')
games.to_csv('Games-Calculated.csv')
!cp Games-Calculated.csv "drive/My Drive/"


In [106]:
#teams.team_name.unique()
#teams.groupby('team_name')['team_name'].nunique()

#games.groupby('home_team')['home_team'].count()
#team_names_only_df = pd.DataFrame(games.home_team)
#team_names_only_df = pd.DataFrame(games.road_team)
#team_names_only_df.shape

df = pd.DataFrame({
    'team_name': games.home_team.append(games.road_team)
    }
)
df2 = teams.merge(df, left_on='team_name', right_on='team_name', how='outer')
unknowns = df2.groupby('team_name')['team_name'].count().reset_index(name='count').sort_values(['count'], ascending=False)

df2[df2['team_name'].str.contains('John')].head(200)

#teams[teams['team_name'].str.contains('Monroe')].head(200)

#games[games['home_team'].str.contains(unknown)]


#not existing: []
# rename: Youngstown St. -> Youngstown State 
# rename: Middle Tenn. -> Middle Tennessee

#renames = [('Youngstown St.', 'Youngstown State'), ('Middle Tenn.', 'Middle Tennessee'), ('Ohio St.', 'Ohio State')]
#for rename in renames:
#  games.loc[(games.home_team == rename[0]),'home_team'] = rename[1]
#  games.loc[(games.road_team == rename[0]),'road_team'] = rename[1]
df2[df2.isna().any(axis=1)]


,conference,team_name
34843,NaN,St. John's (NY)
34844,NaN,St. John's (NY)
34845,NaN,St. John's (NY)
34846,NaN,St. John's (NY)
34847,NaN,St. John's (NY)
...,...,...
47037,NaN,Lincoln (Pa.)
47038,NaN,John. & Wales (R.I.)
47039,NaN,John. & Wales (R.I.)
47040,NaN,Skidmore
